In [2]:
from itertools import combinations
import pandas as pd
import numpy as np
from math import *
from decimal import Decimal 
from numpy.linalg import norm
import os

In [3]:
print (os.getcwd())
os.chdir('D:/OneDrive/ASU/2021 Spring/Applied Project/ASU_Applied_Project_2021/Data')
print (os.getcwd())

C:\Users\Jinhang Jiang
D:\OneDrive\ASU\2021 Spring\Applied Project\ASU_Applied_Project_2021\Data


In [8]:
data = pd.read_csv("fanbaseover.csv")

In [26]:
# definie class fan_overlap
class fan_overlap:
    
    # Class instantiation, col1 would be node also our target, col2 would be edge  
    def __init__ (self, data, col1, col2):
        self.data = data
        self.Celebrity = col1
        self.Usernames = col2
    
    #find the number of unique celebrities and users
    def unique_obs(self):
        return print("Number of Unique Celebrity:",len(self.Celebrity.unique()),"\n"
                     "Number of Users:",len(self.Usernames),"\n"
                     "Number of Unique Users:",len(self.Usernames.unique()),"\n"
                     "Unique Ratio for Users: {:.2%}".format((len(self.Usernames.unique())/len(self.Usernames))))
    
    #find the frequency table. note: when col = 1, we get frequency table for col1;
    def cel_freq(self, col):
        if col == 1:
            freq = self.Celebrity.value_counts()
        else:
            freq = self.Usernames.value_counts()
        return freq
    
    #find the unique names of Celebrities
    def get_names(self, col):
        if col == 1:
            names = self.Celebrity.unique()
        else:
            names = self.Usernames.unique()
        return names
    
    #find the number of shared fans for any two celebrities
    def overlaps2(self, cel1, cel2):
        l = [cel1,cel2]
        Res = pd.DataFrame(self.data.loc[self.data.iloc[:,0].isin(l)].groupby(self.data.iloc[:,1])[data.columns[0]].nunique().eq(len(l)))
        num_of_overlaps = len(Res[Res.iloc[:,0]==True])
        return int(num_of_overlaps)
    
    #find the number of shared fans for any three celebrities
    def overlaps3(self, cel1, cel2, cel3):
        l = [cel1,cel2, cel3]
        Res = pd.DataFrame(self.data.loc[self.data.iloc[:,0].isin(l)].groupby(self.data.iloc[:,1])[data.columns[0]].nunique().eq(len(l)))
        num_of_overlaps = len(Res[Res.iloc[:,0]==True])
        return int(num_of_overlaps)
    
    #find the number of possible combinations of the nodes(Celebrity), r is the number of nodes in one combination
    def nCr(self, r): 
        n = len(self.Celebrity.unique())
        def fact(n): 
            res = 1
            for i in range(2, n+1): 
                res = res * i 
            return res 
        return print("The Number of Possible Combinations is:",int((fact(n) / (fact(r)* fact(n - r)))))
    
    #generate the cross table of fan base overlap
    def crosstable(self):
        s = pd.crosstab(self.Celebrity,self.Usernames)
        s = s.dot(s.T)
        out = s.mask(np.triu(np.ones(s.shape)).astype(bool)).stack()
        return out
    
    #generate adjacency matrix
    def adjacency_matrix(self):
        #matrix = pd.crosstab(self.Usernames,self.Celebrity)
        df_merge = self.data.merge(self.data, on='Usernames')
        results = pd.crosstab(df_merge.Celebrity_x, df_merge.Celebrity_y)
        np.fill_diagonal(results.values, 0)
        network_table=results
        return network_table
    
    #generate transition matrix
    def transition_matrix(self):
        matrix = pd.crosstab(self.Usernames,self.Celebrity)
        cel_matrix = np.asmatrix(matrix)
        cel_matrix_transpose = cel_matrix.transpose()
        final_matrix = cel_matrix_transpose.dot(cel_matrix)
        transition = final_matrix/final_matrix.sum(axis=0)
        transitionMatrix = transition.T
        TM = pd.DataFrame(transitionMatrix)
        TM.columns = matrix.columns
        TM.index = matrix.columns
        return TM
    
    #generate MCL convergence, get the steady state. The default iteration_count = 20
    def convergence(self):
        #create transition matrix
        matrix = self.crosstable()
        
        p_transition = self.transition_matrix()
        n_states = p_transition.shape[0]
        A = np.append(
            arr=p_transition.T - np.eye(n_states),
            values=np.ones(n_states).reshape(1, -1),
            axis=0
        )
        
        # Moore-Penrose pseudoinverse = (A^TA)^{-1}A^T
        pinv = np.linalg.pinv(A)
        pinv1 = pinv.T[-1]
        #create dictionary for the final results
        test_values= pinv1.tolist()
        test_keys = self.get_names(1)
        convergence_dict = {test_keys[i]: test_values[i] for i in range(len(test_keys))}
        return {key : round(convergence_dict[key], 6) for key in convergence_dict} 
    
    #generate minkowksi distance, r = 1 manhattan, r = 2 euclidean. Default = 2
    def cos_sim(self, x, y):
        neww=pd.crosstab(data.Celebrity,data.Usernames)
        a = neww.loc[x]
        b = neww.loc[y]
        dist = (a @ b.T) / (norm(a)*norm(b))
        return dist
    
    #generate a list of most dissimilar celebrities, default is 3
    def best_complete(self,cel,n = 3):
        cel = cel
        CelebrityDistances = []
        for i in model.get_names(1):
            if i != cel:
                temp = (i,model.cos_sim(cel,i))
                CelebrityDistances.append(tuple(temp))
        def Sort_list_tuples(tup):  
            return(sorted(tup, key = lambda x: x[1]))  
        CelebritySortedDistances = []
        CelebritySortedDistances = Sort_list_tuples(CelebrityDistances)
        CelebrityXNN = ""
        tuple_list = sorted(CelebrityDistances, key=lambda t: t[1], reverse=False)[:n]
        CelebrityXNN = [a_tuple[0] for a_tuple in tuple_list]
        return CelebrityXNN
    
    #generate a list of the most similar celebrities for target,the default is 3
    def best_sub(self,cel,n=3):
        cel = cel
        CelebrityDistances = []
        for i in model.get_names(1):
            if i != cel:
                temp = (i,model.cos_sim(cel,i))
                CelebrityDistances.append(tuple(temp))
        def Sort_list_tuples(tup):  
            return(sorted(tup, key = lambda x: x[1])) 
        CelebritySortedDistances = []
        CelebritySortedDistances = Sort_list_tuples(CelebrityDistances)
        CelebrityXNN = ""
        tuple_list = sorted(CelebrityDistances, key=lambda t: t[1], reverse=True)[:n]
        CelebrityXNN = [a_tuple[0] for a_tuple in tuple_list]
        return CelebrityXNN

In [27]:
model = fan_overlap(data, data.Celebrity,data.Usernames)
model.unique_obs()

Number of Unique Celebrity: 8 
Number of Users: 6432 
Number of Unique Users: 5474 
Unique Ratio for Users: 85.11%


In [28]:
model.cel_freq(1)

NCT          2095
BTS          1168
BlackPink     853
iZone         687
GFriend       637
Naeun Son     443
Solar         378
Seolhyun      171
Name: Celebrity, dtype: int64

In [29]:
model.get_names(1)

array(['BTS', 'BlackPink', 'GFriend', 'NCT', 'Naeun Son', 'Seolhyun',
       'Solar', 'iZone'], dtype=object)

In [30]:
print(model.overlaps2('iZone','BlackPink'))
print(model.overlaps2('iZone','GFriend'))

88
135


In [31]:
print(model.overlaps3('Seolhyun','Naeun Son',"Solar"))
print(model.overlaps3('iZone','BTS','GFriend'))

20
40


In [32]:
model.crosstable()

Celebrity  Celebrity
BlackPink  BTS           63.0
GFriend    BTS           75.0
           BlackPink     83.0
NCT        BTS           95.0
           BlackPink     91.0
           GFriend      115.0
Naeun Son  BTS           39.0
           BlackPink     58.0
           GFriend       77.0
           NCT           65.0
Seolhyun   BTS           23.0
           BlackPink     42.0
           GFriend       48.0
           NCT           49.0
           Naeun Son     40.0
Solar      BTS           56.0
           BlackPink     55.0
           GFriend       79.0
           NCT          104.0
           Naeun Son     50.0
           Seolhyun      38.0
iZone      BTS           69.0
           BlackPink     88.0
           GFriend      135.0
           NCT          127.0
           Naeun Son     67.0
           Seolhyun      49.0
           Solar         74.0
dtype: float64

In [33]:
model.nCr(2)

The Number of Possible Combinations is: 28


In [34]:
model.adjacency_matrix()

Celebrity_y,BTS,BlackPink,GFriend,NCT,Naeun Son,Seolhyun,Solar,iZone
Celebrity_x,,,,,,,,
BTS,0,63,75,95,39,23,56,69
BlackPink,63,0,83,91,58,42,55,88
GFriend,75,83,0,115,77,48,79,135
NCT,95,91,115,0,65,49,104,127
Naeun Son,39,58,77,65,0,40,50,67
Seolhyun,23,42,48,49,40,0,38,49
Solar,56,55,79,104,50,38,0,74
iZone,69,88,135,127,67,49,74,0


In [36]:
model.transition_matrix()
#model.transition_matrix()

Celebrity,BTS,BlackPink,GFriend,NCT,Naeun Son,Seolhyun,Solar,iZone
Celebrity,,,,,,,,
BTS,0.735516,0.039673,0.047229,0.059824,0.024559,0.014484,0.035264,0.043451
BlackPink,0.047262,0.639910,0.062266,0.068267,0.043511,0.031508,0.041260,0.066017
GFriend,0.060048,0.066453,0.510008,0.092074,0.061649,0.038431,0.063251,0.108086
NCT,0.034659,0.033200,0.041955,0.764320,0.023714,0.017877,0.037942,0.046333
Naeun Son,0.046484,0.069130,0.091776,0.077473,0.528010,0.047676,0.059595,0.079857
Seolhyun,0.050000,0.091304,0.104348,0.106522,0.086957,0.371739,0.082609,0.106522
Solar,0.067146,0.065947,0.094724,0.124700,0.059952,0.045564,0.453237,0.088729
iZone,0.053241,0.067901,0.104167,0.097994,0.051698,0.037809,0.057099,0.530093


In [37]:
model.convergence()

{'BTS': 0.153578,
 'BlackPink': 0.128917,
 'GFriend': 0.120793,
 'NCT': 0.265087,
 'Naeun Son': 0.081141,
 'Seolhyun': 0.044487,
 'Solar': 0.080658,
 'iZone': 0.125338}

In [40]:
model.best_sub("NCT",n=1)

['Solar']

In [41]:
model.best_complete("NCT",n=1) #most dissimilar

['BTS']

In [42]:
model.best_sub("Naeun Son",n=1)

['Seolhyun']

In [43]:
model.best_complete("Naeun Son",n=1) 

['BTS']

In [191]:
CelebrityDistances = []
target_cel = 'Naeun Son'
for i in model.get_names(1):
    if i != target_cel:
        temp = (i,model.cos_sim(target_cel,i))
        CelebrityDistances.append(tuple(temp))
CelebrityDistances

[('BlackPink', 0.1393770409172632),
 ('Seolhyun', 0.20257904182092132),
 ('Solar', 0.18592789348192545),
 ('GFriend', 0.2044626197073549),
 ('iZone', 0.2279299525221145),
 ('BTS', 0.11153977882757872),
 ('NCT', 0.14159846508095772)]